In [ ]:
import warnings
warnings.filterwarnings('ignore')
import glob
from sigpyproc.readers import FilReader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import dates
import matplotlib.dates as mdates
from matplotlib.ticker import AutoMinorLocator
from datetime import datetime, timedelta
import astropy.units as u
from astropy.time import Time
from astropy.visualization import ImageNormalize, PercentileInterval
from astropy.io import fits as pyfits
import matplotlib as mpl
from sunpy.net import attrs as a
from radiospectra.spectrogram import Spectrogram

mpl.rcParams['date.epoch'] = '1970-01-01T00:00:00' # use precise epoch
try:
    mdates.set_epoch('1970-01-01T00:00:00')
except:
    pass

plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['savefig.facecolor'] = 'white'

path = '/home/mnedal/data/I-LOFAR'
stokes = 'I'
mydatetime = '2025-10-06T08:50:00'

In [ ]:
def get_ilofar_file(paths, datetime):
    """
    Find the full path of the I-LOFAR file 
    with a given datetime in a list of paths.
    datetime -> YYYY-mm-ddTHH:MM:SS
    """
    for p in paths:
        parts = p.split('_')
        try:
            dt = p.split('_')[1]
        except ValueError:
            continue
        if datetime == dt:
            return p
    return None

In [ ]:
def freq_axis(freqs):
    """
    Introduce gaps in the frequency axis of I-LOFAR REALTA data.
    """
    gap1 = np.flipud(freqs[288]+(np.arange(59)*0.390625))
    gap2 = np.flipud(freqs[88]+(np.arange(57)*0.390625))
    ax_shape = 59+57-1
    new_freq = np.zeros(ax_shape+freqs.shape[0])
    
    new_freq[0:88] = freqs[0:88]
    new_freq[88:145]  = gap2[:57]
    new_freq[145:345] = freqs[88:288]
    new_freq[345:404] = gap1[:59]
    new_freq[404:] = freqs[289:]
    
    return new_freq

In [ ]:
def df_limits(df):
    vals = df.values.ravel()
    vmin = np.nanpercentile(vals, 5)
    vmax = np.nanpercentile(vals, 99.7)
    return vmin, vmax

In [ ]:
def spec_limits(radiospec):
    vals = radiospec.data.ravel()
    vmin = np.nanpercentile(vals, 5)
    vmax = np.nanpercentile(vals, 98.7)
    return vmin, vmax

In [ ]:
ilofar_paths = sorted(glob.glob(f'{path}/*.fil'))
print(*ilofar_paths, sep='\n')

In [ ]:
filename = get_ilofar_file(ilofar_paths, mydatetime)
print(filename)

a = FilReader(filename) # header
header = a.header.to_dict()

In [ ]:
tstart_obs_str = Time(a.header.tstart, format='mjd').iso
n_samples      = a.header.nsamples
print(tstart_obs_str, n_samples, sep='\n')

data = a.read_block(start=0, nsamps=n_samples)
print(data.shape)

In [ ]:
# making time axis
tstart = Time(data.header.tstart, format='mjd')                    # tstart.iso will tell the time in format yyyy-mm-dd hh:mm:ss
tarray = tstart + (np.arange(data.shape[1])*data.header.tsamp*u.s) # making the time array for realta time resolution
print(len(tarray), tarray[0].iso, tarray[-1].iso, sep='\n')

In [ ]:
dt = datetime.strptime(tarray[1].iso, '%Y-%m-%d %H:%M:%S.%f') - datetime.strptime(tarray[0].iso, '%Y-%m-%d %H:%M:%S.%f')
print('Time cadence:', dt.total_seconds()*1000, 'ms.')

In [ ]:
# Converting the array to datetime object
Tarray = [datetime.strptime(t.iso, '%Y-%m-%d %H:%M:%S.%f') for t in tarray]
print(Tarray[0], Tarray[-1], sep='\n')

In [ ]:
# export the frequency axis
freqs = data.header.chan_freqs
print(freqs[0], freqs[-1], sep='\n')

In [ ]:
new_freq = freq_axis(freqs)

data = np.log10(data)
data[np.where(np.isinf(data)==True)] = 0.0

data2 = np.empty((new_freq.shape[0], data.shape[1]))    
data2[:] = np.NaN
data2[0:88] = data[0:88]
data2[145:345] = data[88:288]
data2[404:] = data[289:]

In [ ]:
freq_mode3 = np.linspace(10, 90, 199)
freq_mode5 = np.linspace(110, 190, 200)
freq_mode7 = np.linspace(210, 270, 88)

df_mode3 = pd.DataFrame(data=data2[404:].T, columns=freq_mode3[::-1])
df_mode5 = pd.DataFrame(data=data2[145:345].T, columns=freq_mode5[::-1])
df_mode7 = pd.DataFrame(data=data2[:88].T, columns=freq_mode7[::-1])

In [ ]:
print(df_mode3.shape, df_mode5.shape, df_mode7.shape, sep='\n')

In [ ]:
# Ensure the 'time' column is in datetime format
df_mode3.index = pd.to_datetime(Tarray)
df_mode5.index = pd.to_datetime(Tarray)
df_mode7.index = pd.to_datetime(Tarray)

In [ ]:
# Save the dataframes as a pickle files
df_mode3.to_pickle(f'{path}/df_mode3_realta_{mydatetime}_S{stokes}.pkl')
df_mode5.to_pickle(f'{path}/df_mode5_realta_{mydatetime}_S{stokes}.pkl')
df_mode7.to_pickle(f'{path}/df_mode7_realta_{mydatetime}_S{stokes}.pkl')

---

## Load the dataframes from the pickle files

In [ ]:
mydatetime = '20261006'
df_mode3 = pd.read_pickle(f'{path}/df_mode3_realta_{mydatetime}_S{stokes}.pkl')
df_mode5 = pd.read_pickle(f'{path}/df_mode5_realta_{mydatetime}_S{stokes}.pkl')
df_mode7 = pd.read_pickle(f'{path}/df_mode7_realta_{mydatetime}_S{stokes}.pkl')

time_mode3 = df_mode3.index
time_mode5 = df_mode5.index
time_mode7 = df_mode7.index

freq_mode3 = df_mode3.columns
freq_mode5 = df_mode5.columns
freq_mode7 = df_mode7.columns

In [ ]:
df_mode3.head()

In [ ]:
# Slice the DataFrame between start_date and end_date
start_date = '2025-10-06 08:55:30'
end_date   = '2025-10-06 09:05:00'

df_mode3_slice = df_mode3.loc[start_date:end_date]
df_mode5_slice = df_mode5.loc[start_date:end_date]
df_mode7_slice = df_mode7.loc[start_date:end_date]

del df_mode3
del df_mode5
del df_mode7

## Resampling to the same cadence of ORFEES

In [ ]:
# Downsample to 1-second resolution for faster testing and visualization
df_mode3_100ms = df_mode3_slice.resample('100ms').mean()
df_mode5_100ms = df_mode5_slice.resample('100ms').mean()
df_mode7_100ms = df_mode7_slice.resample('100ms').mean()

In [ ]:
# remove the const background
mode3_new = df_mode3_100ms.values - np.tile(np.nanmean(df_mode3_100ms.values,0), (df_mode3_100ms.values.shape[0],1))
mode5_new = df_mode5_100ms.values - np.tile(np.nanmean(df_mode5_100ms.values,0), (df_mode5_100ms.values.shape[0],1))
mode7_new = df_mode7_100ms.values - np.tile(np.nanmean(df_mode7_100ms.values,0), (df_mode7_100ms.values.shape[0],1))

# construct dataframes again
df_mode3_new = pd.DataFrame(data=mode3_new, columns=freq_mode3)
df_mode5_new = pd.DataFrame(data=mode5_new, columns=freq_mode5)
df_mode7_new = pd.DataFrame(data=mode7_new, columns=freq_mode7)

# set the time column as the dataframe index
df_mode3_new.index = pd.to_datetime(df_mode3_100ms.index)
df_mode5_new.index = pd.to_datetime(df_mode5_100ms.index)
df_mode7_new.index = pd.to_datetime(df_mode7_100ms.index)

del mode3_new
del mode5_new
del mode7_new

In [ ]:
use_limits = True

fig, ax = plt.subplots(figsize=[12,5])

for df in [df_mode5_new, df_mode7_new]:
    if use_limits:
        vmin, vmax = df_limits(df)
        ax.pcolormesh(
            df.index,
            df.columns,
            df.values.T,
            cmap='RdYlBu_r',
            vmin=vmin,
            vmax=vmax
        )
    else:
        ax.pcolormesh(
            df.index,
            df.columns,
            df.values.T,
            cmap='RdYlBu_r'
        )
ax.xaxis.set_minor_locator(AutoMinorLocator(n=3))
ax.yaxis.set_minor_locator(AutoMinorLocator(n=4))
ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.set_xlim(pd.Timestamp('2025-10-06T08:57'), pd.Timestamp('2025-10-06T09:02'))
ax.set_ylim(df_mode7_new.columns[0], df_mode5_new.columns[-1])
fig.tight_layout()
plt.show()

In [ ]:
(df_mode5_new.index[1] - df_mode5_new.index[0]).total_seconds()

## Plot ORFEES

In [ ]:
data_dir = '/home/mnedal/data/ORFEES'
mydate = '2025-10-06'
year, month, day = mydate.split('-')
files = sorted(glob.glob(f'{data_dir}/*{year}{month}{day}_*.fts'))
print(*files, sep='\n')

orfees = pyfits.open(files[0])
orfees_i = np.hstack([orfees[2].data[f'STOKESI_B{i}'] for i in range(1, 6)]).T
data = orfees_i.T
print(data.shape)

# Remove the background by taking the data from the quiet background and divide it by the data
new_data = data - np.tile(np.nanmean(data,0), (data.shape[0],1))

new_data = new_data.T
orfees_time_str = orfees[0].header['DATE-OBS']
orfees_times = Time(orfees_time_str) + (orfees[2].data['TIME_B1']/1000)*u.s # times are not the same for all sub spectra
orfees_freqs = np.hstack([orfees[1].data[f'FREQ_B{i}'] for i in range(1, 6)])*u.MHz

orfees_meta = {
    'observatory': orfees[0].header['ORIGIN'],
    'instrument': orfees[0].header['INSTRUME'],
    'detector': orfees[0].header['INSTRUME'],
    'freqs': orfees_freqs.reshape(-1),
    'times': orfees_times,
    'wavelength': a.Wavelength(orfees_freqs[0,0], orfees_freqs[0,-1]),
    'start_time': orfees_times[0],
    'end_time': orfees_times[-1]
}

orfees_spec_i = Spectrogram(new_data, orfees_meta)

In [ ]:
use_limits = True

fig, ax = plt.subplots(figsize=[12,5])

times = orfees_spec_i.times.datetime
freqs = orfees_spec_i.frequencies.value
data  = orfees_spec_i.data

if use_limits:
    vmin, vmax = spec_limits(orfees_spec_i)
    ax.pcolormesh(times, freqs, data, cmap='RdYlBu_r', vmin=vmin, vmax=vmax)
else:
    ax.pcolormesh(times, freqs, data, cmap='RdYlBu_r')

ax.xaxis.set_minor_locator(AutoMinorLocator(n=3))
ax.yaxis.set_minor_locator(AutoMinorLocator(n=4))
ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.set_xlim(pd.Timestamp('2025-10-06T08:57'), pd.Timestamp('2025-10-06T09:02'))
ax.set_ylim(bottom=300, top=orfees_freqs.reshape(-1)[0].value)
fig.tight_layout()
plt.show()

In [ ]:
(times[1] - times[0]).total_seconds()

## Plot I-LOFAR with ORFEES

In [ ]:
use_limits = True

fig, ax = plt.subplots(figsize=[12,10])

for df in [df_mode5_new, df_mode7_new]:
    if use_limits:
        ax.pcolormesh(
            df.index,
            df.columns,
            df.values.T,
            cmap='RdYlBu_r',
            vmin=df_limits(df)[0],
            vmax=df_limits(df)[1]
        )
        ax.pcolormesh(times, freqs, data, cmap='RdYlBu_r',
                      vmin=spec_limits(orfees_spec_i)[0],
                      vmax=spec_limits(orfees_spec_i)[1])
    else:
        ax.pcolormesh(
            df.index,
            df.columns,
            df.values.T,
            cmap='RdYlBu_r'
        )
        ax.pcolormesh(times, freqs, data, cmap='RdYlBu_r')

ax.xaxis.set_minor_locator(AutoMinorLocator(n=3))
ax.yaxis.set_minor_locator(AutoMinorLocator(n=4))
ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.set_xlim(pd.Timestamp('2025-10-06T08:57'), pd.Timestamp('2025-10-06T09:02'))
# ax.set_ylim(bottom=300, top=df_mode5_new.columns[-1])
fig.tight_layout()
plt.show()

In [ ]:
df.index[0], df.index[-1]

In [ ]:
times[0], times[-1]

In [ ]:
df_mode7_new.columns[-1]